In [14]:
import import_ipynb
import DiscModel as DM
import LossFunctions as LF
import GeneralFunctions as GF
import numpy as np

from scipy.interpolate import interp1d
import scipy.integrate as integrate




In [15]:
def Interpolation(i, SampleSizeFactor, Function, DiscModel):
    """
    Interpolates the given function at index i using linear interpolation.

    Parameters:
    i (int): The index of the function to interpolate.
    SampleSizeFactor (float): The factor to scale the grid size by.
    Function (function): The function to interpolate.

    Returns:
    interpNdiff (function): The interpolated function.

    """
    ZGrid_i = DM.ZArray(DiscModel)[i]

    # Create an extended grid for interpolation
    ZGridExtropolated = SampleSizeFactor * np.array(range(ZGrid_i))

    # Perform linear interpolation
    interpNdiff = interp1d(ZGridExtropolated, Function(DiscModel)[i], kind='linear', fill_value='extrapolate')

    return interpNdiff


In [16]:



def Loss_Function_Electron_Array(E, i, DiscModel):
    """
    Calculate the total electron loss function for a given energy and radial index.

    Parameters:
    E (float): The energy value.
    i (int): The index of the radius.

    Returns:
    array: The total electron loss function for electron energy E at radial index i.

    """
    # Get the necessary data from the DiscModel object
    fH = DM.fH(DiscModel, i)
    fH2 = DM.fH2(DiscModel, i)
    fHe = DM.fHe(DiscModel, i)
    fHE = DM.fHE(DiscModel, i)
    DensityH2 = DM.DensityH2(DiscModel)[i][1:][::-1]
    DensityH = DM.DensityH(DiscModel)[i][1:][::-1]
    DensityHe = DM.DensityHe(DiscModel)[i][1:][::-1]
    DensityE = DM.DensityE(DiscModel)[i][1:][::-1]
    Te = DM.Temperature(DiscModel)[i][1:][::-1]

    # Calculate the electron loss function components
    LElectronH = LF.LElectronH(E)
    LElectronH2 = LF.LElectronH2(E)
    LElectronHe = LF.LElectronHe(E)
    LCOULOMBElectron = LF.LCOULOMBElectron(E, DensityH2, DensityH, DensityHe, DensityE, Te)

    # Calculate the total electron loss function
    total_lossfunction = fH * LElectronH + fH2 * LElectronH2 + fHe * LElectronHe + fHE * LCOULOMBElectron

    return total_lossfunction

def Loss_Function_Proton_Array(E, i, DiscModel):
    """
    Calculate the total proton loss function for a given energy and radial index.

    Parameters:
    E (float): The energy value.
    i (int): The index of the radius.

    Returns:
    array: The total electron loss function for electron energy E at radial index i.

    """
    # Get the necessary data from the DiscModel object
    fH = DM.fH(DiscModel, i)
    fH2 = DM.fH2(DiscModel, i)
    fHe = DM.fHe(DiscModel, i)
    fHE = DM.fHE(DiscModel, i)
    DensityH2 = DM.DensityH2(DiscModel)[i][1:][::-1]
    DensityH = DM.DensityH(DiscModel)[i][1:][::-1]
    DensityHe = DM.DensityHe(DiscModel)[i][1:][::-1]
    DensityE = DM.DensityE(DiscModel)[i][1:][::-1]
    Te = DM.Temperature(DiscModel)[i][1:][::-1]

    # Calculate the electron loss function components
    LProtonH = LF.LProtonH(E)
    LProtonH2 = LF.LProtonH2(E)
    LProtonHe = LF.LProtonHe(E)
    LCOULOMBProton = LF.LCOULOMBProton(E, DensityH2, DensityH, DensityHe, DensityE, Te)

    # Calculate the total electron loss function
    total_loss_function = fH * LProtonH + fH2 * LProtonH2 + fHe * LProtonHe + fHE * LCOULOMBProton

    return total_loss_function


In [17]:
def Loss_Function_Electron_2DArray(E_array, i, DiscModel):
    
    LElectronArray = np.array([Loss_Function_Electron_Array(E, i, DiscModel) for E in E_array])
    return np.transpose(LElectronArray)[::-1]

def Loss_Function_Proton_2DArray(E_array,i, DiscModel):
    
    LProtonArray = np.array([Loss_Function_Proton_Array(E, i, DiscModel) for E in E_array])
    return np.transpose(LProtonArray)[::-1]







In [18]:
def RElectron2DArray(E_Interpolation_array, i, DiscModel):
    """
    Calculate the R(E) values for electrons for a given energy array and radial index.

    Parameters:
    E_array (numpy.ndarray): Array of energy values.
    i (int): The index of the radial position.

    Returns:
    numpy.ndarray: Array of R(E) values for electrons.

    """
    LElectronArray = Loss_Function_Electron_2DArray(E_Interpolation_array, i, DiscModel)

    def RElectron1DArray(E):
        """
        Calculate the R(E) value for a given energy for electrons.

        Parameters:
        E (float): The energy value.

        Returns:
        float: The R(E) value for electrons.

        """
        RElectron = []
        for j in range(len(LElectronArray)):
            interp1dLElectron = interp1d(E_Interpolation_array, LElectronArray[j], fill_value='extrapolate')

            ElisteInteg = np.logspace(-1, np.log10(E), len(LElectronArray))
            Integrande = 1 / interp1dLElectron(ElisteInteg)

            RElectronj = integrate.trapezoid(Integrande, x=ElisteInteg, axis=0)
            RElectron.append(RElectronj)
        return np.array(RElectron)

    return RElectron1DArray(E_Interpolation_array)


def RProton2DArray(E_Interpolation_array, i, DiscModel):
    """
    Calculate the R(E) values for protons for a given energy array and radial index.

    Parameters:
    E_array (numpy.ndarray): Array of energy values.
    i (int): The index of the radial position.

    Returns:
    numpy.ndarray: Array of R(E) values for protons.

    """
    LProtonArray = Loss_Function_Proton_2DArray(E_Interpolation_array, i, DiscModel)

    def RProton1DArray(E):
        """
        Calculate the R(E) value for a given energy for protons.

        Parameters:
        E (float): The energy value.

        Returns:
        float: The R(E) value for protons.

        """
        RProton = []
        for j in range(len(LProtonArray)):
            interp1dLProton = interp1d(E_Interpolation_array, LProtonArray[j], fill_value='extrapolate')

            ElisteInteg = np.logspace(-1, np.log10(E), len(LProtonArray))
            Integrande = 1 / interp1dLProton(ElisteInteg)

            RProtonj = np.trapezoid(Integrande, x=ElisteInteg, axis=0)
            RProton.append(RProtonj)
        return np.array(RProton)

    return RProton1DArray(E_Interpolation_array)

In [19]:
'''
import matplotlib.pyplot as plt
DiscModelName="ProDiMoModels/CIW98_XN_AH_CIK"

DiscModel=DM.readDiscModel(DiscModelName)
E_array=np.logspace(-1,9)
E_Interpolation_array=np.logspace(-1, 10, 50)

#LElectronArrayRev=LElectronArray[::-1]
LElectronArray = Loss_Function_Electron_2DArray(E_array, 15,DiscModel)
for j in range(len(LElectronArray)):
    if j % 2 == 0:
        logN=np.log10(DM.ColumnDensityArray(DiscModel)[15,j])
        if logN> 19:
            plt.plot(E_array, LElectronArray[j], label=str(round(logN,2)))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Energy')
plt.ylabel('LElectron')
plt.title('LElectron vs Energy')
plt.legend()
plt.show()


#RElectronArray = RElectron2DArray(E_array, 15, DiscModel)
RProtonArray = RElectron2DArray(E_array, 15, DiscModel)

for j in range(len(RProtonArray)):
    if j % 2 == 0:
        logN=np.log10(DM.ColumnDensityArray(DiscModel)[15,j])
        if logN> 19:
            plt.plot(E_array, RProtonArray[j], label=str(round(logN,2)))

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Energy [eV]')
plt.ylabel('R(E) [cm^2]')
plt.show()


'''


'\nimport matplotlib.pyplot as plt\nDiscModelName="ProDiMoModels/CIW98_XN_AH_CIK"\n\nDiscModel=DM.readDiscModel(DiscModelName)\nE_array=np.logspace(-1,9)\nE_Interpolation_array=np.logspace(-1, 10, 50)\n\n#LElectronArrayRev=LElectronArray[::-1]\nLElectronArray = Loss_Function_Electron_2DArray(E_array, 15,DiscModel)\nfor j in range(len(LElectronArray)):\n    if j % 2 == 0:\n        logN=np.log10(DM.ColumnDensityArray(DiscModel)[15,j])\n        if logN> 19:\n            plt.plot(E_array, LElectronArray[j], label=str(round(logN,2)))\nplt.xscale(\'log\')\nplt.yscale(\'log\')\nplt.xlabel(\'Energy\')\nplt.ylabel(\'LElectron\')\nplt.title(\'LElectron vs Energy\')\nplt.legend()\nplt.show()\n\n\n#RElectronArray = RElectron2DArray(E_array, 15, DiscModel)\nRProtonArray = RElectron2DArray(E_array, 15, DiscModel)\n\nfor j in range(len(RProtonArray)):\n    if j % 2 == 0:\n        logN=np.log10(DM.ColumnDensityArray(DiscModel)[15,j])\n        if logN> 19:\n            plt.plot(E_array, RProtonArray[j]